## Imports

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('ship').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv',inferSchema=True,header=True)

In [4]:
data.columns

Out[16]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

## Using String Indexer to transform string

In [5]:
from pyspark.ml.feature import StringIndexer

In [6]:
si = StringIndexer(inputCol='Cruise_line',outputCol='indexed_cruise')
si.setHandleInvalid('error')

Out[18]: StringIndexer_20a645b23dce

In [7]:
model = si.fit(data)

In [8]:
new_data = model.transform(data)

In [9]:
new_data.head(1)

Out[22]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, indexed_cruise=16.0)]

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
new_data.columns

Out[25]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'indexed_cruise']

## Processing data

In [12]:
assembler = VectorAssembler(inputCols=['indexed_cruise','Age','Tonnage','passengers','length','cabins','passenger_density'],
                           outputCol='features')

In [13]:
assembled_data = assembler.transform(new_data)

In [14]:
assembled_data.head(1)

Out[29]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, indexed_cruise=16.0, features=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [15]:
final_data = assembled_data.select('features','crew')

In [16]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 112|
 mean| 8.001071428571432|
 stddev|3.6749746232237404|
 min| 0.59|
 max| 21.0|
+-------+------------------+

## Training and Evaluating the data

In [18]:
from pyspark.ml.regression import LinearRegression

In [19]:
lr = LinearRegression(labelCol='crew')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
test_result = lr_model.evaluate(test_data)

In [22]:
test_result.residuals.show()

+--------------------+
 residuals|
+--------------------+
-0.31190489809777056|
 -0.4039794181118843|
 -1.7276189824927641|
 -0.908188562389169|
 -0.5628369522497731|
 -1.060684481668039|
 -0.6162025101181126|
 0.5197508532590494|
 0.4523307168787092|
-0.07376793472325716|
-0.18799126917014952|
-0.17607578647636934|
 0.9730665765518101|
 0.17691061662923957|
 0.3424400305151436|
 0.12729458349748946|
 -1.2877484452344774|
 -0.7690555391839968|
 -1.0271823676452403|
 0.19729784581308785|
+--------------------+
only showing top 20 rows

In [23]:
test_result.meanSquaredError

Out[42]: 0.8279689190888991

In [24]:
test_result.rootMeanSquaredError

Out[43]: 0.909927974671017

In [25]:
test_result.r2

Out[44]: 0.9074474656137376

In [26]:
test_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[0.0,7.0,158.0,43...|13.6|
[0.0,10.0,90.09,2...|8.58|
[0.0,15.0,78.491,...| 6.6|
[0.0,21.0,73.941,...|8.22|
[0.0,25.0,73.192,...|8.08|
[1.0,12.0,88.5,21...| 9.3|
[1.0,14.0,101.509...|10.0|
[1.0,15.0,70.367,...| 9.2|
[1.0,28.0,46.052,...| 6.6|
[2.0,9.0,116.0,26...|11.0|
[2.0,10.0,91.6270...| 9.0|
[2.0,11.0,91.6270...| 9.0|
[2.0,11.0,108.977...|12.0|
[2.0,12.0,108.865...|11.0|
[2.0,14.0,30.2769...|3.73|
[2.0,15.0,108.806...|11.1|
[3.0,5.0,86.0,21....| 8.0|
[3.0,11.0,85.0,18...| 8.0|
[3.0,14.0,63.0,14...|5.61|
[3.0,16.0,59.652,...|6.44|
+--------------------+----+
only showing top 20 rows

## Creating unlabeled data to test model

In [27]:
unlabeled_data = test_data.select('features')

In [28]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[0.0,7.0,158.0,43...|
[0.0,10.0,90.09,2...|
[0.0,15.0,78.491,...|
[0.0,21.0,73.941,...|
[0.0,25.0,73.192,...|
[1.0,12.0,88.5,21...|
[1.0,14.0,101.509...|
[1.0,15.0,70.367,...|
[1.0,28.0,46.052,...|
[2.0,9.0,116.0,26...|
[2.0,10.0,91.6270...|
[2.0,11.0,91.6270...|
[2.0,11.0,108.977...|
[2.0,12.0,108.865...|
[2.0,14.0,30.2769...|
[2.0,15.0,108.806...|
[3.0,5.0,86.0,21....|
[3.0,11.0,85.0,18...|
[3.0,14.0,63.0,14...|
[3.0,16.0,59.652,...|
+--------------------+
only showing top 20 rows

In [29]:
result = lr_model.transform(unlabeled_data)

In [30]:
result.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[0.0,7.0,158.0,43...| 13.91190489809777|
[0.0,10.0,90.09,2...| 8.983979418111884|
[0.0,15.0,78.491,...| 8.327618982492764|
[0.0,21.0,73.941,...| 9.12818856238917|
[0.0,25.0,73.192,...| 8.642836952249773|
[1.0,12.0,88.5,21...| 10.36068448166804|
[1.0,14.0,101.509...|10.616202510118113|
[1.0,15.0,70.367,...| 8.68024914674095|
[1.0,28.0,46.052,...|6.1476692831212905|
[2.0,9.0,116.0,26...|11.073767934723257|
[2.0,10.0,91.6270...| 9.18799126917015|
[2.0,11.0,91.6270...| 9.17607578647637|
[2.0,11.0,108.977...| 11.02693342344819|
[2.0,12.0,108.865...| 10.82308938337076|
[2.0,14.0,30.2769...|3.3875599694848564|
[2.0,15.0,108.806...| 10.97270541650251|
[3.0,5.0,86.0,21....| 9.287748445234477|
[3.0,11.0,85.0,18...| 8.769055539183997|
[3.0,14.0,63.0,14...| 6.637182367645241|
[3.0,16.0,59.652,...|6.2427021541869125|
+--------------------+------------------+
only showing top 20 rows

In [31]:
test_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[0.0,7.0,158.0,43...|13.6|
[0.0,10.0,90.09,2...|8.58|
[0.0,15.0,78.491,...| 6.6|
[0.0,21.0,73.941,...|8.22|
[0.0,25.0,73.192,...|8.08|
[1.0,12.0,88.5,21...| 9.3|
[1.0,14.0,101.509...|10.0|
[1.0,15.0,70.367,...| 9.2|
[1.0,28.0,46.052,...| 6.6|
[2.0,9.0,116.0,26...|11.0|
[2.0,10.0,91.6270...| 9.0|
[2.0,11.0,91.6270...| 9.0|
[2.0,11.0,108.977...|12.0|
[2.0,12.0,108.865...|11.0|
[2.0,14.0,30.2769...|3.73|
[2.0,15.0,108.806...|11.1|
[3.0,5.0,86.0,21....| 8.0|
[3.0,11.0,85.0,18...| 8.0|
[3.0,14.0,63.0,14...|5.61|
[3.0,16.0,59.652,...|6.44|
+--------------------+----+
only showing top 20 rows

### Printing predicted vs original for comparison